In [1]:
import glob
import pickle
from sklearn.model_selection import StratifiedKFold

In [2]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import train_test_split
train_data = pd.read_csv("../data/train.csv")
# Binning the numeric variable to different categories
target_std=train_data['target'].std()
max_target=train_data['target'].max()+1
min_target=train_data['target'].min()-1
bins=[min_target,-3*target_std,-2*target_std,-1*target_std,target_std*1,target_std*2,target_std*3,max_target]
labels=[-4,-3,-2,0,2,3,4]
train_data['target_bin']=pd.cut(train_data['target'],bins=bins,labels=labels).astype(int)
train_data['target_bin']=train_data['target_bin'].abs()
train_data.groupby(['target_bin']).size()

target_bin
0    191093
2      8200
3       363
4      2261
dtype: int64

In [11]:
target_std=train_data['target'].std()
target_std

3.850499995486051

In [12]:
[min_target,-3*target_std,-2*target_std,-1*target_std,target_std*1,target_std*2,target_std*3,max_target]

[-34.219280949999998,
 -11.551499986458154,
 -7.700999990972102,
 -3.850499995486051,
 3.850499995486051,
 7.700999990972102,
 11.551499986458154,
 18.9650684]

In [ ]:
#     param = {'num_leaves': 300,
#              'min_data_in_leaf': 100,
#              'objective': 'regression',
#              'max_depth': 15,
#              'learning_rate': 0.01,
#              "boosting": "gbdt",
#              "feature_fraction": 0.9,
#              "bagging_freq": 1,
#              "bagging_fraction": 0.8,
#              "bagging_seed": 11,
#              "metric": 'rmse',
#              # "lambda_l1": 0.2634,
#              "random_state": 133,
#              "verbosity": -1}


# {
# 'filepath': '../Fresh\\active_first_diff.csv',
#  'best_iteration0': 484,
#  'best_score0': 3.6632739775611216,
#  'best_iteration1': 330,
#  'best_score1': 3.6985657675999226,
#  'best_iteration2': 357,
#  'best_score2': 3.6788080841478394,
#  'best_iteration3': 552,
#  'best_score3': 3.651419743890834,
#  'best_iteration4': 517,
#  'best_score4': 3.6491765269973442
# }


In [8]:
def run_lgb(train_X, train_y, val_X, val_y):
    param = {'num_leaves': 300,
             'min_data_in_leaf': 100,
             'objective': 'regression',
             'max_depth': 15,
             'learning_rate': 0.01,
#              "boosting": "gbdt",
             "boosting": "rf",
             "feature_fraction": 0.9,
             "bagging_freq": 1,
             "bagging_fraction": 0.8,
             "bagging_seed": 11,
             "metric": 'rmse',
             # "lambda_l1": 0.2634,
             "random_state": 133,
             "verbosity": -1}

    lgtrain = lgb.Dataset(train_X, label=train_y)
    lgval = lgb.Dataset(val_X, label=val_y)
    evals_result = {}
    model = lgb.train(param, lgtrain, 2000, valid_sets=[lgval], early_stopping_rounds=100, verbose_eval=100,
                      evals_result=evals_result)
    return model, evals_result

In [4]:
def model_run(filepath,train_data1):
    results={}
    results['filepath']=filepath[0]
    for file in filepath:
        data_features = pd.read_csv(file)
        train_data1=pd.merge(train_data1,data_features,how='left',on='card_id')
    output_feature=['target']
    input_features=[x for x in train_data1.columns if x not in ['first_active_month', 'card_id','target','target_bin']]
    train_X = train_data1[input_features]
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    train_X = train_X.select_dtypes(include=numerics)
    train_y = train_data1[output_feature]
    kf = StratifiedKFold(n_splits=5, random_state=2018, shuffle=True)
    counter=0
    for dev_index, val_index in kf.split(train_X,train_data1['target_bin']):
        
        dev_X, val_X = train_X.loc[dev_index, :], train_X.loc[val_index, :]
        dev_y, val_y = train_y.loc[dev_index], train_y.loc[val_index]
        model, evals_result = run_lgb(dev_X, dev_y, val_X, val_y)
        results['best_iteration'+str(counter)]=model.best_iteration
        results['best_score'+str(counter)]=model.best_score['valid_0']['rmse']
        counter+=1
    return results

In [9]:
filelist14=glob.glob("../Fresh/*.csv")
pickle.dump( filelist14, open( "filelist14.p", "wb" ) )
print(filelist14)
results=model_run(filelist14,train_data)


['../Fresh\\active_first_diff.csv', '../Fresh\\card_new_mercid.csv', '../Fresh\\card_summary_future.csv', '../Fresh\\card_summary_history.csv', '../Fresh\\category1_allN.csv', '../Fresh\\category1_allY.csv', '../Fresh\\category2_all0.0.csv', '../Fresh\\category2_all1.0.csv', '../Fresh\\category2_all2.0.csv', '../Fresh\\category2_all3.0.csv', '../Fresh\\category2_all4.0.csv', '../Fresh\\category2_all5.0.csv', '../Fresh\\category3_allA.csv', '../Fresh\\category3_allB.csv', '../Fresh\\category3_allC.csv', '../Fresh\\diffhistfuture.csv', '../Fresh\\future_length.csv', '../Fresh\\hist_length.csv', '../Fresh\\merchant_joint_analysis.csv', '../Fresh\\most_recent_purchases_range_all0.csv', '../Fresh\\most_recent_purchases_range_allA.csv', '../Fresh\\most_recent_purchases_range_allB.csv', '../Fresh\\most_recent_purchases_range_allC.csv', '../Fresh\\most_recent_purchases_range_allD.csv', '../Fresh\\most_recent_purchases_range_allE.csv', '../Fresh\\most_recent_sales_range_allA.csv', '../Fresh\\mo

In [7]:
results

{'filepath': '../Fresh\\active_first_diff.csv',
 'best_iteration0': 484,
 'best_score0': 3.6632739775611216,
 'best_iteration1': 330,
 'best_score1': 3.6985657675999226,
 'best_iteration2': 357,
 'best_score2': 3.6788080841478394,
 'best_iteration3': 552,
 'best_score3': 3.651419743890834,
 'best_iteration4': 517,
 'best_score4': 3.6491765269973442}